<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 03**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**



*   Revisar o proceso de análise léxica
*   Introduzir o processo de análise sintática
*   Implementar analisadores sintáticos usando a ferramenta rply



**REVISÃO DO PROCESSO DE ANÁLISE LÉXICA**

Nas nossas aulas passadas, trabalhamos com a gramática abaixo, que permite reconhecer expressões aritméticas:

\<expression\> ::= NUMBER

               | \<expression\> "+" \<expression\>

               | \<expression\> "-" \<expression\>

               | \<expression\> "*" \<expression\>

               | \<expression\> "/" \<expression\>

               | "(" <expression> ")"

Para usar o módulo rply, primeiro instalamos o pacote no nosso notebook:



In [ ]:
!pip install rply

O segundo passo foi construir um analisador léxico, que quebrava as nossas expressões em tokens:

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

Para mostrar somente os tokens reconhecidos de uma expressão, podemos utilizar o código abaixo:

In [ ]:
for token in lexer.lex('1+1-1'):
  print(token)

**PROCESSO DE ANÁLISE SINTÁTICA**

A análise sintática é o segundo passo no processo de compilação:

<img src="https://www.researchgate.net/profile/Nuno-Oliveira-15/publication/266497079/figure/fig1/AS:295651775664128@1447500284302/Common-Compiler-Phases.png"> <img>

Um **analisador sintático** recebe uma sequencia de tokens identificados pelo **analisador léxico** e verifica se estes tokens estão na ordem correta, segundo a **gramática** da linguagem que está sendo compilada.

O produto final do processo de análise sintática é uma **árvore n-ária** chamada **árvore de análise sintática**. 

<img src="https://i.stack.imgur.com/woqkC.png"> <img>

Cada nó desta árvore sintática armazena uma parte reconhecida da entrada, de acordo com a gramática fornecida:

\<expression\> ::= NUMBER

               | \<expression\> "+" \<expression\>

               | \<expression\> "-" \<expression\>

               | \<expression\> "*" \<expression\>

               | \<expression\> "/" \<expression\>

               | "(" <expression> ")"


Normalmente, para cada linha da gramática, criamos uma classe para representar tal linha:



In [ ]:
from rply.token import BaseBox

class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def eval(self):
        return self.value

class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()

class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval()


Uma vez que temos como representar cada linha a nossa gramática, dentro de cada nó, observem que temos um método eval. Este método irá calcular o valor representado pot aquele nó. Posteriormente iremos acrescentar novos métodos para, por exemplo, verificar tipos, gerar código, dentre outros.

Finalmente, temos a implementação do analisador sintático: 



In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()


Como resultado, temos a árvore sintática. Percorrendo esta árvore, podemos calcular o valor da expressão usando o método eval().

In [ ]:
parser.parse(lexer.lex('1 + 1')).eval()

**EXERCÍCIO**

Modifique a gramática abaixo para permitir atribuições do tipo x=1+2*3:

\<expression\> ::= NUMBER

           | \<expression\> "+" \<expression\>
 
           | \<expression\> "-" \<expression\>
 
           | \<expression\> "*" \<expression\>
 
           | \<expression\> "/" \<expression\>
 
           | "(" <expression> ")"

*digite sua solução aqui*

**EXERCÍCIO**

Implemente um analisador sintático para a gramática modificada do exercício anterior:


In [ ]:
#digite sua aolução aqui

**ATIVIDADE EAD**

Implemente um analisador sintático para a gramática modificada do exercício anterior:

1.   Modifique a gramática anterior para permitir expressões envolvendo exponenciação (ˆ). Por exemplo: x = 2ˆ3.
2.   Implente um analisador sintático para reconhecer ou recusar operações de exponenciação.

